In [0]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd

In [0]:
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.applications.resnet50 import ResNet50 
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from PIL import Image
import base64
import random
from keras.models import load_model
import cv2
from io import BytesIO

In [0]:
Image=[299,299]

In [0]:
train_path = '/content/drive/My Drive/DataSets/Face Recognition/Training'
valid_path = '/content/drive/My Drive/DataSets/Face Recognition/Test/'

resnet = ResNet50(input_shape=Image + [3], weights='imagenet', include_top=False)

for layer in resnet.layers:
  layer.trainable = False  

x = Flatten()(resnet.output)
x = Dense(1000, activation='relu')(x)
prediction = Dense(2, activation='softmax')(x)

model = Model(inputs=resnet.input, outputs=prediction)
model.summary()
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [0]:
from keras.preprocessing.image import ImageDataGenerator,array_to_img,img_to_array,load_img
training_data_gen=ImageDataGenerator(rotation_range=40,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            rescale=1./255,
                            fill_mode="nearest")

test_data_gen=ImageDataGenerator(rescale=1.0/255)

training_set = training_data_gen.flow_from_directory('/content/drive/My Drive/DataSets/Face Recognition/Training',
                                                 target_size = (299, 299),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')

test_set = test_data_gen.flow_from_directory('/content/drive/My Drive/DataSets/Face Recognition/Test',
                                            target_size = (299, 299),
                                            batch_size = 16,
                                            class_mode = 'categorical')



In [0]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=2,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

In [0]:
from keras.models import load_model

model.save('facerecognition.h5')

In [0]:
model=load_model("/content/facerecognition.h5")

In [0]:
import cv2
photo = cv2.imread("photo.jpg")
photo=cv2.resize(photo,(299,299))
fullbody_classifier=cv2 .CascadeClassifier("/content/drive/My Drive/CNN Dataset/haarcascade_frontalface_default.xml")
faces=fullbody_classifier.detectMultiScale(photo,1.3,2)
for x,y,w,h in faces:
    face = photo[y:y+h, x:x+w]
    if type(face) is np.ndarray:
      face = cv2.resize(face, (299,299))
      im = Image.fromarray(face, 'RGB')
      img_array = np.array(im)
      img_array = np.expand_dims(img_array, axis=0)
      pred = model.predict(img_array)          
      name=""
      if(pred[0][0]>0.5):
        name='ASHWINI'
        cv2.putText(photo,name,(x,y), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow(photo)

      elif(pred[0][1]>0.5):
        name='KASHISH'
        cv2.putText(photo,name,(x,y), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow(photo)

    else:
      print("No face found")


    cv2.destroyAllWindows()